In [23]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity


class CollabRecommendation:
    def __init__(self, user_data, product_data, rating_data):
        self.user_data = user_data
        self.product_data = product_data
        self.rating_data = rating_data

    def collaborative_filtering_recommendations(self, target_user_id, top_n=10):
        # Pivot the data to create a user-item matrix
        user_item_matrix =self.rating_data.pivot_table(index='userid', columns='productid', values='rating', aggfunc='mean').fillna(0)

        # Get the target user's ratings
        target_user_ratings = user_item_matrix.loc[target_user_id]

        # Find users who have rated items in common with the target user
        similar_users = user_item_matrix[(user_item_matrix > 0).any(axis=1) & (user_item_matrix.index != target_user_id)]

        # Calculate the cosine similarity between the target user and similar users
        similar_users_similarity = cosine_similarity(target_user_ratings.values.reshape(1, -1), similar_users.values)

        # Sort similar users by similarity score
        similar_users_indices = similar_users_similarity.argsort()[0][::-1]

        # Generate recommendations
        recommended_items = []
        for user_index in similar_users_indices[:top_n]:
            similar_user_ratings = similar_users.iloc[user_index]
            recommended_items.extend(similar_user_ratings[similar_user_ratings > 0].index)

        # Remove items already rated by the target user
        recommended_items = [item for item in recommended_items if item not in target_user_ratings.index]
        # Get details of recommended items
        recommended_items_details = product_data[product_data['id'].isin(recommended_items)][['id', 'name']]
        return recommended_items_details

    def recommend(self, user_id, top_n=20):
        return self.collaborative_filtering_recommendations(user_id, top_n=top_n)

    def save(self, filepath):
        """Save the model to a file."""
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load(filepath):
        """Load the model from a file."""
        with open(filepath, 'rb') as f:
            return pickle.load(f)


# Example usage
if __name__ == "__main__":
    # Load data
    user_data = pd.read_csv('user_data.csv')
    ratings_data = pd.read_csv('ratings_test.csv')
    product_data = pd.read_csv('test_data.csv')

    # Initialize the model
    model = CollabRecommendation(user_data, product_data, ratings_data)

    # Example user ID
    user_id = '35262'

    # Generate recommendations
    recommendations = model.recommend(user_id, top_n=10)

    print(f"Recommendations for user ID {user_id}:")
    for rec in recommendations:
        print(f"Product ID: {rec[0]}, Product Name: {rec[1]}")


Recommendations for user ID 35262:
Product ID: i, Product Name: d
Product ID: n, Product Name: a
